In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/road-accidents-2019-2023/vehicles-2020.csv
/kaggle/input/road-accidents-2019-2023/char-2019.csv
/kaggle/input/road-accidents-2019-2023/char-2021.csv
/kaggle/input/road-accidents-2019-2023/place-2022.csv
/kaggle/input/road-accidents-2019-2023/users-2023.csv
/kaggle/input/road-accidents-2019-2023/users-2021.csv
/kaggle/input/road-accidents-2019-2023/vehicles-2021.csv
/kaggle/input/road-accidents-2019-2023/char-2020.csv
/kaggle/input/road-accidents-2019-2023/place-2019.csv
/kaggle/input/road-accidents-2019-2023/users-2019.csv
/kaggle/input/road-accidents-2019-2023/place-2021.csv
/kaggle/input/road-accidents-2019-2023/users-2020.csv
/kaggle/input/road-accidents-2019-2023/char-2023.csv
/kaggle/input/road-accidents-2019-2023/users-2022.csv
/kaggle/input/road-accidents-2019-2023/vehicles-2022.csv
/kaggle/input/road-accidents-2019-2023/vehicles-2019.csv
/kaggle/input/road-accidents-2019-2023/place-2020.csv
/kaggle/input/road-accidents-2019-2023/place-2023.csv
/kaggle/input/road-a

In [2]:
%%time
## all imports
import polars as pl
import pyarrow.parquet as pq
import dask.dataframe as dd
import os
import shutil
import json
from enum import Enum
from datetime import datetime
from ydata_profiling import ProfileReport
from pathlib import Path
import random

#For excel stuff
import openpyxl
from openpyxl.drawing.image import Image


# Visualization
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px

# Concurrency
from concurrent.futures import ThreadPoolExecutor, ProcessPoolExecutor, as_completed
import time

#  Warnings
import warnings
warnings.filterwarnings('ignore')

# Set random state
random_state = 42
# Set figure size
plt.rcParams["figure.figsize"] = (20, 20)


## Set the static file locactions
filepaths = {'vehicles':
    {
    2019: '/kaggle/input/road-accidents-2019-2023/vehicles-2019.csv',
    2020: '/kaggle/input/road-accidents-2019-2023/vehicles-2020.csv',
    2021: '/kaggle/input/road-accidents-2019-2023/vehicles-2021.csv',
    2022: '/kaggle/input/road-accidents-2019-2023/vehicles-2022.csv',
    2023: '/kaggle/input/road-accidents-2019-2023/vehicles-2023.csv',
    },
    'users':{
        2019: '/kaggle/input/road-accidents-2019-2023/users-2019.csv',
        2020: '/kaggle/input/road-accidents-2019-2023/users-2020.csv',
        2021: '/kaggle/input/road-accidents-2019-2023/users-2021.csv',
        2022: '/kaggle/input/road-accidents-2019-2023/users-2022.csv',
        2023: '/kaggle/input/road-accidents-2019-2023/users-2023.csv'
    },
    'places': {
        2019: '/kaggle/input/road-accidents-2019-2023/place-2019.csv',
        2020: '/kaggle/input/road-accidents-2019-2023/place-2020.csv',
        2021: '/kaggle/input/road-accidents-2019-2023/place-2021.csv',
        2022: '/kaggle/input/road-accidents-2019-2023/place-2022.csv',
        2023: '/kaggle/input/road-accidents-2019-2023/place-2023.csv'
    },
    'characteristics':{
        2019: '/kaggle/input/road-accidents-2019-2023/char-2019.csv',
        2020: '/kaggle/input/road-accidents-2019-2023/char-2020.csv',
        2021: '/kaggle/input/road-accidents-2019-2023/char-2021.csv',
        2022: '/kaggle/input/road-accidents-2019-2023/char-2022.csv',
        2023: '/kaggle/input/road-accidents-2019-2023/char-2023.csv'
    }
             
}

class ExtensionMethods:
    @staticmethod
    def generate_filename(filename=None,extension=None):
        current_datetime = datetime.now()
        f = current_datetime.strftime("%Y_%m_%d_%H%M")
        if (filename is None) or (extension is None):
            return str(f)
        else:
            stitched_f = str(filename)+"_"+str(f)+"."+str(extension)
            return str(stitched_f)

    @staticmethod
    def get_file_name_without_extension(filename):
        if filename == None:
            return "Provide a file"
        return Path(filename).stem
        
        

CPU times: user 4.57 s, sys: 720 ms, total: 5.29 s
Wall time: 8.32 s


In [3]:
class CategoryBaseEnum(Enum):
    @classmethod
    def Name(cls):
        return f"{cls.__name__}"
    @classmethod
    def IsCategory(cls):
        return(True if len(cls.__members__)>0 else False)
    @classmethod
    def get_description(cls):
        return cls.__doc__ or "No description available"
    @classmethod
    def to_dict(cls):
        return {member.name:member.value for member in cls}
    @classmethod
    def to_json(cls, indent=4):
        return json.dumps(cls.to_dict(), indent=indent)

In [4]:
class FileTypeEnum(Enum):
    @classmethod
    def Name(cls):
        return f"{cls.__name__}"
    @classmethod
    def get_description(cls):
        return cls.__doc__ or "No description available"
    @classmethod
    def to_dict(cls):
        return {
            member.name:member.value.to_dict() for member in cls}

In [5]:
%%time
class Vehicle_Enum(FileTypeEnum):
    '''Collection of the categories for file type 'vehicle.csv' '''

    class Num_Acc(CategoryBaseEnum):
        '''The Index/Number of the Crash follows the pattern yyyyxxxxx and is the index column''' 
    class id_vehicule(CategoryBaseEnum):
        '''The vehicle id  in terms of xxx-xxx'''
    class num_veh(CategoryBaseEnum):
        '''The Number of the vehicle category , something in french '''

    class senc(CategoryBaseEnum): # senc DIRECTION OF TRAFFIC
        '''The Direction of Traffic '''
        UNKNOWN = 0 
        PK_INCREASING = 1 #PK ascending or address ascending
        PK_DECREASING = 2
        NO_REFERENCE_POINT = 3
        RANDOM = -1 ##TODO VERIFY THIS
    
    class catv(CategoryBaseEnum): # catv Vehicle Category
        '''The Category of Vehicle involved in the crash'''
        UNDETERMINED = 0
        BICYCLE = 1
        MOPED_LESS_EQUAL_50CC = 2
        MICROCAR = 3
        TOURISM_VEHICLE = 7
        UTILITY_VEHICLE_PTAC_1_5T_3_5T = 10 #1.5<PTAC<3.5
        HEAVY_TRUCK_PTAC_3_5T_7_5T = 13
        HEAVY_TRUCK_PTAC_OVER_7_5T = 14
        HEAVY_TRUCK_OVER_3_5T_WITH_TRAILER = 15
        TRACTOR_ONLY = 16
        TRACTOR_WITH_SEMI_TRAILER = 17
        SPECIAL_VEHICLE = 20
        AGRICULTURAL_TRACTOR = 21
        SCOOTER_LESS_EQUAL_50CC = 30
        MOTORCYCLE_50CC_125CC = 31
        SCOOTER_50CC_125CC = 32
        MOTORCYCLE_OVER_125CC = 33
        SCOOTER_OVER_125CC = 34
        LIGHT_QUAD_LESS_EQUAL_50CC = 35
        HEAVY_QUAD_OVER_50CC = 36
        BUS = 37
        COACH = 38
        TRAIN = 39
        TRAMWAY = 40
        THREE_WHEELED_VEHICLE_LESS_EQUAL_50CC = 41
        THREE_WHEELED_VEHICLE_50CC_125CC = 42
        THREE_WHEELED_VEHICLE_OVER_125CC = 43
        MOTORIZED_PERSONAL_TRANSPORT = 50
        NON_MOTORIZED_PERSONAL_TRANSPORT = 60
        OTHER_VEHICLE = 99

    class obs(CategoryBaseEnum):  #obs Static ObstacleHit
        ''' The Static/Stationary Obstacle Hit'''
        UNKNOWN = 0
        PARKED_VEHICLE = 1
        TREE_ON_ROADSIDE = 2
        METAL_BARRIER = 3
        CONCRETE_BARRIER = 4
        OTHER_BARRIER=5
        BUILDING_WALL_BRIDGE_PIER = 6 #BUILDING, WALL OR BRIDGE PIER
        VERTICAL_SIGNPOST_OR_EMERGENCY_CALLBOX = 7
        POLE = 8
        URBAN_FURNITURE = 9 # NO CLUE WHAT URBAN FURNITURE
        PARAPET = 10
        REFUGE_ISLAND_BOLLARD = 11 #THE ROAD ISLAND / BOLLARD
        SIDEWALK = 12 #SIDEWALK OR CURB
        DITCH = 13 #DITCH OR EMBANKMENT
        OTHER_OBS_ON_ROADWAY = 14
        OTHER_OBS_ON_SIDEWALK = 15
        ROADWAY_EXIT_WITOUT_OBSTACLES = 16
        AQUEDUCT_HEAD = 17

    class obsm(CategoryBaseEnum): #obsm #Mobile obstacle hit
        '''The Dynamic Obstacle Hit'''
        UNKNOWN = 0
        PEDESTRIAN =1 
        VEHICLE = 2
        RAIL_VEHICLE = 3
        ANIMAL_DOMESTIC = 4
        ANIMAL_WILD = 5
        OTHER = 9
        
    class choc(CategoryBaseEnum): #choc Initial Point of Impact
        '''The Initial Point of Impact of the crash'''
        UNKNOWN = 0
        FRONT = 1
        FRONT_LEFT = 2
        FRONT_RIGHT = 3
        REAR = 4
        REAR_RIGHT = 5
        REAR_LEFT = 6
        SIDE_LEFT = 7
        SIDE_RIGHT = 8
        MULTIPLE = 9

    
    class manv(CategoryBaseEnum): #manv , Main action before crash
        '''The Main Action performed by the user before the crash'''
        UNKNOWN = 0
        CIRCULATING_NO_DIRECTION_CHANGE =1
        CIRCULATING_SAME_DIRECTION = 2 #SAME DIRECTION SAME LANE
        CIRCULATION_BETWEEN_2_LANES = 3
        CIRCULATING_REVERSING = 4
        CIRCULATING_AGAINTS_FLOW_TRAFFIC = 5
        CIRCULATING_CROSSING_MEDIAN_STRIP = 6
        CIRCULATING_IN_BUSLANE_SAME_DIRECTION = 7
        CIRCULATING_IN_BUSLANE_OPP_DIRECTION = 8
        CIRCULATING_INSERTION =9
        CIRCULATING_TURNING_AROUND_CARRIAGE_WAY = 10
        CHANGING_LANE_LEFT = 11
        CHANGING_LANE_RIGHT = 12
        DEPORT_LEFT = 13
        DEPORT_RIGHT = 14
        TURNING_LEFT = 15
        TURNING_RIGHT = 16
        OVERTAKING_LEFT = 17
        OVERTAKING_RIGHT = 18 
        #VARIOUS S
        CROSSING_ROAD = 19
        PARKING_ACTION = 20
        AVOIDANCE_ACTION = 21
        DOOR_OPENED = 22
        STOP_NO_PARKING = 23
        PARKED_WITH_PASS = 24 # PARKED WITH PASSANGERS
        DRIVING_SIDEWALK = 25
        OTHER_ACTIONS = 26
        
    
    class motor(CategoryBaseEnum): # motor
        '''The Type of Motor(In terms of fuel type) involved in the crash'''
        UNKNOWN = 0
        CONVENTIONAL_FUEL = 1 # PETROL, DIESEL ,ETC
        HYBRID_ELECTRIC = 2
        ELECTRIC = 3
        HYDROGEN = 4
        HUMAN_POWERED = 5
        OTHER = 6

    #occ : Number of occupants in the Public transport (bus) including the driver irrespective if they were injured or not
    class occutc(CategoryBaseEnum):
        '''Number of occupants in the Public transport (bus) including the driver irrespective if they were injured or not'''


CPU times: user 2.21 ms, sys: 35 µs, total: 2.24 ms
Wall time: 2.11 ms


In [6]:
'''These are dummy inplace enum definitions for the corresponding enums '''
class User_Enum(FileTypeEnum):
    '''User stuff''' 
        
class Place_Enum(FileTypeEnum):
    '''Place Enum'''

class Character_Enum(FileTypeEnum):
    '''Characteristic Enum'''

In [7]:
class RoadAccident_Enum(Enum):
    '''This is the meta enum containing filetype enums'''
    VEHICLES = Vehicle_Enum
    USERS = User_Enum
    PLACE = Place_Enum
    CHARACTER = Character_Enum
    
    @classmethod
    def to_dict(cls):
        return {en.name: en.value.to_dict() for en in cls}
    @classmethod
    def to_json(cls,indent=4):
        return json.dumps(cls.to_dict(), indent=indent)
    @classmethod
    def save_json(cls):
        _data = cls.to_json()
        filepath = os.path.join(os.getcwd(),ExtensionMethods.generate_filename("road_accidents_","json"))
        file = open(filepath, "w")
        file.write(_data)
        file.close()
        print(f"The data has been printed to {filepath}")
        

In [8]:
a = RoadAccident_Enum.to_dict()
b = a['VEHICLES']['catv']
b

{'UNDETERMINED': 0,
 'BICYCLE': 1,
 'MOPED_LESS_EQUAL_50CC': 2,
 'MICROCAR': 3,
 'TOURISM_VEHICLE': 7,
 'UTILITY_VEHICLE_PTAC_1_5T_3_5T': 10,
 'HEAVY_TRUCK_PTAC_3_5T_7_5T': 13,
 'HEAVY_TRUCK_PTAC_OVER_7_5T': 14,
 'HEAVY_TRUCK_OVER_3_5T_WITH_TRAILER': 15,
 'TRACTOR_ONLY': 16,
 'TRACTOR_WITH_SEMI_TRAILER': 17,
 'SPECIAL_VEHICLE': 20,
 'AGRICULTURAL_TRACTOR': 21,
 'SCOOTER_LESS_EQUAL_50CC': 30,
 'MOTORCYCLE_50CC_125CC': 31,
 'SCOOTER_50CC_125CC': 32,
 'MOTORCYCLE_OVER_125CC': 33,
 'SCOOTER_OVER_125CC': 34,
 'LIGHT_QUAD_LESS_EQUAL_50CC': 35,
 'HEAVY_QUAD_OVER_50CC': 36,
 'BUS': 37,
 'COACH': 38,
 'TRAIN': 39,
 'TRAMWAY': 40,
 'THREE_WHEELED_VEHICLE_LESS_EQUAL_50CC': 41,
 'THREE_WHEELED_VEHICLE_50CC_125CC': 42,
 'THREE_WHEELED_VEHICLE_OVER_125CC': 43,
 'MOTORIZED_PERSONAL_TRANSPORT': 50,
 'NON_MOTORIZED_PERSONAL_TRANSPORT': 60,
 'OTHER_VEHICLE': 99}